# Imports

In [ ]:
import os
from dotenv import load_dotenv
import warnings

import pandas as pd
import numpy as np

from sqlalchemy import create_engine, text as sql_text

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from geopy.distance import geodesic

# Config

In [ ]:
# Warnings
warnings.filterwarnings("ignore")

# .env loading
dotenv_path = "../.env"
load_dotenv(dotenv_path)

# DB connection
DB_CONFIG = {
    "user": os.getenv("DB_DESTINATION_USER"),
    "pwd": os.getenv("DB_DESTINATION_PASSWORD"),
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "db": os.getenv("DB_DESTINATION_NAME")
}

CONNECTION_STR = "postgresql://{}:{}@{}:{}/{}".format(
    DB_CONFIG["user"],
    DB_CONFIG["pwd"],
    DB_CONFIG["host"],
    DB_CONFIG["port"],
    DB_CONFIG["db"]
)

ENGINE = create_engine(
    CONNECTION_STR + "?options=--transaction-mode=autocommits?slmode=disable",
    isolation_level=None
)

TARGET = "price"

# Style and output options
sns.set_theme(style="white")

pd.options.display.max_rows = 1024
pd.options.display.max_columns = 1024
pd.set_option("display.max_colwidth", 128)


print(DB_CONFIG)
print(CONNECTION_STR)
print(ENGINE)

# Functions

In [ ]:
def df_quik_view(df, drop2dubs=None, asc=False):
    unique_values = [
        df[col].sort_values(ascending=asc).astype(str).unique()
        for col in df.columns
    ]
    duplicates_num = [
        df[col].duplicated(keep=False).sum()
        for col in df.columns
    ]
    nans = df.isna().sum()

    table_scan = pd.DataFrame(
        {
            "values_num":     df.count(),
            "nulls_num":      nans,
            "completeness":   100 - (nans / (nans + df.notna().sum()) * 100),
            "uniques_num":    df.astype(str).nunique(),
            "unique_values":  unique_values,
            "duplicates_num": duplicates_num,
            "dtype":          df.dtypes
        }
    )

    df_shape = df.shape
    duplicates = df.duplicated(
        subset=df.columns.drop(drop2dubs) if drop2dubs is not None else None
    ).sum()
    memory = df.memory_usage(index=False).sum() / 1_000_00

    print("General data analysis:")
    print()
    print("Shape of the table:     ", df_shape)
    print("Duplicates in the table: {}, ({}%)"
          .format(duplicates, round(duplicates / df_shape[0], 4) * 100)
          +(f" (without {drop2dubs})" if drop2dubs is not None else " ")
         )
    print("Memory usage:            " + str(round(memory, 1)) + "+ MB")
    print()

    return table_scan


def plot_boxplot(data, feature, target=None):
    plt.figure(figsize=(10, 6))
    
    if target:
        sns.boxplot(data=data, x=target, y=feature, fliersize=5, linewidth=2)
    else:
        sns.boxplot(data=data, y=feature, fliersize=5, linewidth=2)
    
    plt.title(f"{feature} distribution")
    plt.xlabel(target if target else '')
    plt.ylabel(feature)
    plt.grid(True)
    plt.gca().set_facecolor('white')
    
    plt.show()


def plot_histogram(data, feature, target=None, bins=100):
    plt.figure(figsize=(10, 6))
    
    if target:
        sns.histplot(data=data.sort_values(by=target), x=feature, hue=target, bins=bins, kde=False, element="bars", alpha=0.8)
    else:
        sns.histplot(data=data, x=feature, bins=bins, kde=False, element="bars", alpha=0.8)
    
    plt.title(f"{feature} distribution")
    plt.xlabel(feature)
    plt.ylabel("Amount")
    plt.grid(True)
    plt.gca().set_facecolor("white")

    plt.show()


def plot_ecdf(data, feature, target=None):
    plt.figure(figsize=(10, 6))
    
    if target:
        sns.ecdfplot(data=data, x=feature, hue=target, alpha=0.8)
    else:
        sns.ecdfplot(data=data, x=feature, alpha=0.8)
    
    plt.title(f"{feature} ECDF")
    plt.xlabel(feature)
    plt.ylabel("Probability")
    plt.grid(True)
    plt.gca().set_facecolor("white")
    
    plt.show()


def plot_scalar_feature(data, feature, target=None, bins=100):
    plot_boxplot(data, feature, target)
    plot_histogram(data, feature, target, bins)
    plot_ecdf(data, feature, target)
    print(f"{feature} statistics:")
    print()
    print(data[feature].describe())


def check_is_inside_mkad(lat, lon, center, radius):
    point = (lat, lon)
    distance = geodesic(center, point).kilometers
    return distance <= radius


def plot_scatter(data, x, y, kde_frac=0.1):
    fig, ax = plt.subplots(figsize=(6, 6))
    sns.scatterplot(data=data, x=x, y=y, s=5, color=".15")
    sns.histplot(data=data, x=x, y=y, bins=50, pthresh=0.1, cmap="mako")
    sns.kdeplot(data=data.sample(frac=kde_frac), x=x, y=y, levels=5,
                color="black", linewidth=1)
    plt.show()


def show_feature_info(data_info, feature):
    print("PARAMETER:", feature, "\n")
    if feature in data_info.columns:
        print(data_info.loc[feature])


def make_scalar_eda(data, feature, data_info):
    show_feature_info(data_info, feature)
    plot_scalar_feature(data, feature)
    # plot_scatter(data, feature, TARGET)


def plot_bar(data, feature, y="id"):
    pivot_data = data.pivot_table(index=feature, values=y, aggfunc="count")
    sns.barplot(pivot_data, x=feature, y=y, label="Amount")
    plt.show()


def make_categorical_eda(data, feature, data_info):
    show_feature_info(data_info, feature)
    plot_bar(data, feature)
    plot_boxplot(data, TARGET, feature)
    plot_histogram(data, TARGET, feature)
    plot_ecdf(data, TARGET, feature)


def get_iqr_bounds(data, col, threshold=1.5):
    IQR = data[col].quantile(0.75) - data[col].quantile(0.25)
    up_bound = data[col].quantile(0.75) + (IQR * threshold)
    low_bound = data[col].quantile(0.25) - (IQR * threshold)

    return up_bound, low_bound

# Data Loading

In [ ]:
query = """
    SELECT *
      FROM yandex_real_estate_data;
"""

data = pd.read_sql_query(
    sql=sql_text(query),
    con=ENGINE.connect()
)

data = data.replace("true", 1).replace("false", 0)

ids_colums = [col for col in data.columns if col.endswith("id")]

data.head()

In [ ]:
data_info = df_quik_view(data, ids_colums)
data_info

# EDA & processing

## Drop duplicates

In [ ]:
data = data.drop_duplicates(
    subset=data.columns.drop(ids_colums)
)
data.shape

## Price (target)

In [ ]:
print("PARAMETER:", TARGET, "\n")
print(data_info.loc[TARGET])
plot_scalar_feature(data, TARGET)

In [ ]:
print("Observation number", data.shape)
data = data[(data[TARGET] >= 1e+6) & (data[TARGET] <= 1.5e+8)]
print("Observations number after discarding observations with extreme prices:", data.shape)

In [ ]:
# plot_scalar_feature(data, TARGET)

## Latitude & longitude

In [ ]:
# # Центр Москвы (Красная площадь)
# center_moscow = (55.751244, 37.618423)
# radius_mkad_km = 20  # Радиус МКАДа в километрах

# data["inside_mkad"] = data.apply(
#     lambda row: check_is_inside_mkad(
#         row["latitude"],
#         row["longitude"],
#         center_moscow,
#         radius_mkad_km
#     ), axis=1
# ).astype(int)
# data["inside_mkad"].value_counts()

In [ ]:
# data = data[data["inside_mkad"] == 1].reset_index(drop=True)
# data.shape

In [ ]:
# fig = px.scatter_mapbox(
#     data, lat="latitude", lon="longitude", color="inside_mkad", zoom=14, height=600)
# fig.update_layout(mapbox_style="open-street-map")
# fig.show()

## latitude

In [ ]:
make_scalar_eda(data, "latitude", data_info)

## longitude

In [ ]:
make_scalar_eda(data, "longitude", data_info)

## build_year

In [ ]:
make_scalar_eda(data, "build_year", data_info)

## building_type_int

In [ ]:
make_categorical_eda(data, "building_type_int", data_info)

## ceiling_height

In [ ]:
make_scalar_eda(data, "ceiling_height", data_info)

## flats_count

In [ ]:
make_scalar_eda(data, "flats_count", data_info)

## floors_total

In [ ]:
make_scalar_eda(data, "floors_total", data_info)

## has_elevator

In [ ]:
make_categorical_eda(data, "has_elevator", data_info)

## floor

In [ ]:
make_scalar_eda(data, "floor", data_info)

## kitchen_area

In [ ]:
make_scalar_eda(data, "kitchen_area", data_info)

## living_area

In [ ]:
make_scalar_eda(data, "living_area", data_info)

## rooms

In [ ]:
make_scalar_eda(data, "rooms", data_info)

## is_apartment

In [ ]:
make_categorical_eda(data, "is_apartment", data_info)

## studio

In [ ]:
print(data["studio"].value_counts(dropna=False))
data = data.drop("studio", axis=1)

## total_area

In [ ]:
make_scalar_eda(data, "total_area", data_info)

# Anomalies visualization

In [ ]:
data["is_anomaly"] = 0
for col in data.columns.drop(ids_colums + ["building_type_int"]):
    up_bound, low_bound = get_iqr_bounds(data, col)
    data["is_anomaly"] = np.where(
        data[col].between(low_bound, up_bound),
        data["is_anomaly"],
        1
    )

print(data.shape)
ds = data.copy()
ds = ds[ds["is_anomaly"] == 0].reset_index(drop=True)
ds.shape

In [ ]:
df_quik_view(ds, ids_colums)

In [ ]:
features2drop = [
    "studio", "is_anomaly", "inside_mkad", "is_apartment", "inside_mkad",
    "is_anomaly", "has_elevator"
]
ds = ds.drop(features2drop, axis=1)

# Corr

In [ ]:
corr_matrix = ds.drop(
    ids_colums + ["building_type_int"],
    axis=1
).corr(method="spearman")

plt.figure(figsize=(10,10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f')
plt.show()